In [1]:
import argparse
import pickle
import numpy as np
from datetime import datetime as dt
import time
import pandas as pd
from beautifultable import BeautifulTable
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.pipeline import make_pipeline

In [2]:
""" Functions for model set up """

def get_features(fmat, label_cols, feature_file=None, n_feats2sel=None):
    all_cols = fmat.columns.values.tolist()
    if not feature_file:
        data_cols = [c for c in all_cols if c not in label_cols]
    else:
        fsel = pd.read_csv(feature_file)
        select_feats = fsel['feature'].head(int(n_feats2sel)).tolist()
        data_cols = [c for c in all_cols if c not in label_cols and c in select_feats]
    return(data_cols)
    
def fmt_data(fmat, subset, label_cols, data_cols, keep_groups=True):
    # get desired subset
    fmat_fmt = fmat[fmat['label'].isin(subset)]
    fmat_fmt.reset_index(inplace=True, drop=True)
    # format data for sklearn input
    X = fmat_fmt[data_cols].to_numpy()
    y = fmat_fmt[label_cols[1]].to_numpy()
    ids = fmat_fmt[label_cols[0]]
    # keep/drop group col option
    if keep_groups:
        groups = fmat_fmt[label_cols[2]].to_numpy()
        return(X, y, ids, groups)
    else:
        return(X, y, ids)
    
def fmt_outdir(outdir):
    if outdir.endswith('/'):
        return(outdir)
    else:
        return(outdir+'/')

def def_grp_split(method='GroupShuffleSplit', num_splits=5, train_size=0.7, seed=None):
    if method == 'GroupShuffleSplit':
        gs = GroupShuffleSplit(n_splits = num_splits, train_size=train_size, random_state=seed)
    elif method == 'GroupKFold':
        gs = GroupKFold(n_splits = num_splits)
    elif method == 'StratifiedGroupKFold':
        gs = StratifiedGroupKFold(n_splits = num_splits)
    else:
        print('Invalid group split strategy specified; please choose one of: "GroupShuffleSplit", "GroupKFold", or "StratifiedGroupKFold"')
        print('Review documentation for more information on each method: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection')
    return(gs)

def load_model(model_file, seed=None):
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
    try:
        n_steps = len(model.named_steps)
        idx = n_steps-1
        model_name = type(model[idx]).__name__
        if seed:
            setattr(model[idx], 'random_state', seed)
    except:
        model_name = type(model).__name__
        if seed:
            try:
                setattr(model, 'random_state', seed)
            except:
                print(f'WARNING: failed to set random state for {model} to {seed}!')
    return(model, model_name)

def split_data(X, y, train_idx, test_idx):
    # get splits
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    y_test = y[test_idx]
    # check split & label balance
    label, counts = np.unique(y_train, return_counts=True)
    label_counts_train = dict(zip(label, counts))
    label, counts = np.unique(y_test, return_counts=True)
    label_counts_test = dict(zip(label, counts))
    print(f' ► # train PPIs = {len(X_train)}')
    print(f' ► train +/- label counts: {label_counts_train}')
    print(f' ► # test PPIs = {len(X_test)}')
    print(f' ► test +/- label counts: {label_counts_test}')
    return(X_train, y_train, X_test, y_test)

def write_results(all_res, test_scores, model_name, outdir, fdr_cutoff):
    # format path, file names
    outdir_fmt = fmt_outdir(outdir)
    fdr_fmt = int(fdr_cutoff*100)
    bases = ['scored_interactions_all', f'scored_interactions_fdr{fdr_fmt}', 'precision_recall' ]
    all_out = outdir_fmt+f'{bases[0]}_{model_name}.csv'
    high_conf_out = outdir_fmt+f'{bases[1]}_{model_name}.csv'
    pr_out = outdir_fmt+f'{bases[2]}_{model_name}.csv'
    # format all scored PPIs
    all_res.sort_values('ppi_score', inplace=True, ascending=False)
    all_res.reset_index(inplace=True, drop=True)
    print(f' ► Writing all scored PPIs to {all_out} ...')
    all_res.to_csv(all_out, index=False)
    # get PR curve & threshold PPIs at given FDR
    pr_curve, high_conf_ppis = threshold_ppis(test_scores, all_res, fdr_cutoff)
    print(f' ► Writing precision-recall results to {pr_out} ...')
    pr_curve.to_csv(pr_out)
    print(f' ► Writing results to {high_conf_out} ...')
    high_conf_ppis.to_csv(high_conf_out, index=False)

""" Functions for model fitting & evaluation """

def calc_pr(df):
    # compute precision/recall
    print(f" ► Computing precision/recall ...")
    tp_count = 0
    fp_count = 0
    p_list = []
    r_list = []
    f_list = []
    all_pos = len(df[df['label'] == 1])
    for i in range(len(df)):
        if df['label'][i] == 1:
            tp_count += 1
        else:
            fp_count += 1
        tps = tp_count
        fps = fp_count
        fns = all_pos - tps
        precision = tps/(tps+fps)
        recall = tps/(tps+fns)
        fdr = 1 - precision
        p_list.append(float(precision))
        r_list.append(float(recall))
        f_list.append(float(fdr))
    df['precision'] = p_list
    df['recall'] = r_list
    df['fdr'] = f_list
    return(df)

def threshold_ppis(test_scores, all_res, fdr_cutoff):
    # compute precision/recall
    test_scores_pr = calc_pr(test_scores)
    thres_df = test_scores_pr[test_scores_pr['fdr'] <= fdr_cutoff]
    prob_cutoff = min(thres_df['ppi_score'])
    print(f' ► PPI score cutoff for {int(fdr_cutoff*100)}% FDR: {prob_cutoff}')
    # theshold results
    thres_df = all_res[all_res['ppi_score'] >= prob_cutoff]
    ids = thres_df['ID'].str.split(' ', expand=True)
    print(f' ► # total PPIs evaluated: {len(all_res)}')
    print(f' ► # PPIs above threshold: {len(thres_df)}')
    try: # only works if there are no self-self PPIs
        uniq_prots = np.unique(ids[[0, 1]].values)
        print(f' ► # unique proteins above threshold: {len(uniq_prots)}')
    except:
        print('WARNING: Problem with IDs detected.')
        print(ids[0:51])
    df_out = thres_df[['ID','ppi_score']]
    return(test_scores_pr, df_out)

In [3]:
featmat = '../ppi_ml/data/featmats/featmat_labeled.pkl'
num_splits = 1
train_size = 0.7
seed = 17
remove_per_step = 3
threads = 12
model = '../ppi_ml/results/tpot/gkfold/tpot_model_3.pkl'
group_split_method = 'GroupShuffleSplit'
fdr_cutoff = 0.1
feature_selection = None
num_feats = None

In [4]:
t0 = time.time()
## read in data & define model params
print(f'[{dt.now()}] Loading feature matrix ...')
with open(featmat, 'rb') as handle:
    fmat = pickle.load(handle)

label_cols = ['ID', 'label', 'super_group']
data_cols = get_features(fmat, label_cols, feature_selection, num_feats)
X, y, ids, groups = fmt_data(fmat, [1,-1], label_cols, data_cols, keep_groups=True)
X_pred, y_pred, ids_pred = fmt_data(fmat, [0], label_cols, data_cols, keep_groups=False)

print(f'[{dt.now()}] Loading complex group split method parameters ...')
gs = def_grp_split(group_split_method, num_splits, train_size, seed)

print(f'[{dt.now()}] Loading model parameters ...')
model, model_name = load_model(model, seed)

table = BeautifulTable()
table.columns.header = ["PARAMETER", "SETTING"]
table.rows.append(["Protein complex split method", group_split_method])
table.rows.append(["# train/test (CV) splits", num_splits])
table.rows.append(["Machine learning method", model_name])
table.rows.append(["% data for training (approx*):", f'{int(train_size*100)}%'])
table.rows.append(["% data for testing (approx*):", f'{int((1-train_size)*100)}%'])
table.rows.append(["% FDR threshold", f'{int(fdr_cutoff*100)}%'])
table.rows.append(["# features to be used", len(data_cols)])

print()
print(f'[{dt.now()}] Pipeline settings detected:')
print(table)
print('*Note: Group split method will attempt to get as close to these settings as possible, but results may vary depending on the seed, group sizes, and number of cross-validation splits specified.')

[2023-03-23 11:59:09.563588] Loading feature matrix ...
[2023-03-23 11:59:42.800752] Loading complex group split method parameters ...
[2023-03-23 11:59:42.801077] Loading model parameters ...

[2023-03-23 11:59:42.820455] Pipeline settings detected:
+--------------------------------+-------------------+
|           PARAMETER            |      SETTING      |
+--------------------------------+-------------------+
|  Protein complex split method  | GroupShuffleSplit |
+--------------------------------+-------------------+
|    # train/test (CV) splits    |         1         |
+--------------------------------+-------------------+
|    Machine learning method     |     LinearSVC     |
+--------------------------------+-------------------+
| % data for training (approx*): |        70%        |
+--------------------------------+-------------------+
| % data for testing (approx*):  |        30%        |
+--------------------------------+-------------------+
|        % FDR threshold         |

/stor/home/rmcox/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator SelectPercentile from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/stor/home/rmcox/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator LinearSVC from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/stor/home/rmcox/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator Pipeline from version 1.2.2 when using version 1.2.1. This might l

In [5]:
def get_scores(model, array, labels, ids):
    # get scores
    scores = model.predict_proba(array)
    probabilities = np.split(scores, 2, axis=1)
    neg_prob = probabilities[0]
    pos_prob = probabilities[1]
    # format into df
    df = pd.DataFrame()
    df['ID'] = ids
    df['label'] = labels
    df['ppi_score'] = pos_prob
    df.sort_values('ppi_score', inplace=True, ascending=False)
    df.reset_index(inplace=True, drop=True)
    return(df)

In [6]:
## fit model, compute precision/recall, and output results
for i, (train_idx, test_idx) in enumerate(gs.split(X, y, groups)):

    # get train/test splits
    print()
    print(f"[{dt.now()}] Getting test/train splits ({i+1}/{num_splits}) ...")
    X_train, y_train, X_test, y_test = split_data(X, y, train_idx, test_idx)

    # fit model
    print(f"[{dt.now()}] Fitting {model_name} ...")
    
    # extract results
    test_ids = ids[test_idx]
    train_ids = ids[train_idx]
    pred_ids = ids_pred

    if len(test_ids) > len(train_ids) and num_splits == 1:
        print("ERROR: Imbalanced train/test split. Try a different seed using the --seed argument.")

    print(f"[{dt.now()}] Getting probability scores ...")
    test_scores = get_scores(model, X_test, y_test, test_ids)
    train_scores = get_scores(model, X_train, y_train, train_ids)
    pred_scores = get_scores(model, X_pred, y_pred, pred_ids)


[2023-03-23 11:59:42.844905] Getting test/train splits (1/1) ...
 ► # train PPIs = 24405
 ► train +/- label counts: {-1: 18276, 1: 6129}
 ► # test PPIs = 2111
 ► test +/- label counts: {-1: 1611, 1: 500}
[2023-03-23 11:59:42.928535] Fitting LinearSVC ...
[2023-03-23 11:59:45.973481] Getting probability scores ...


In [7]:
X_test

array([[ 9.9110e-01,  4.6291e+00, -1.7700e-02, ...,  5.4000e-01,
         4.3660e-01,  7.4130e-01],
       [ 9.9720e-01,  6.0172e+00, -5.8400e-02, ...,  1.6800e-02,
        -1.3500e-02,  6.9760e-01],
       [ 9.3580e-01,  8.2533e+00, -2.4000e-02, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       ...,
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  2.2500e-02,
        -6.4000e-03,  7.5360e-01],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  1.6800e-02,
         7.4000e-03,  6.7870e-01],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00]])